In [1]:
from graded_asignments.graded_2.MLPEnsemble import MLPEnsemble
from graded_asignments.graded_2.DTForest import DTForest
from graded_asignments.graded_2.utils import *
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from statistics import mean

#### 5a)
*Load the file “data_2022.csv” and create a test set in the same format as the training set you created in (2.a)*

In [2]:
pd_2021 = pd.read_csv('data/data_2021.csv')
pd_2022 = pd.read_csv('data/data_2022.csv')
np_2021 = pd_2021['Demand'].to_numpy()
np_2022 = pd_2022['Demand'].to_numpy()

#### 5b)
*Pick a model from (2, 3, or 4), and use this year’s demand so far to predict the missing values from 26.08.2022 to 31.12.2022. Save the predictions in a list (5 pts)*

In [21]:
forest_size = 200
window_size = 10
x, y = slide(np_2021, window_size)
single_tree = DecisionTreeRegressor()
dt_forest = DTForest(forest_size)
mlp_forest = MLPEnsemble(forest_size)

In [22]:
test_data, train_data = split([pd_2021, pd_2022])

In [5]:
single_tree.fit(x, y)  # training single tree

DecisionTreeRegressor()

In [6]:
dt_forest.train(train_data, window_size)

training...........................................................................................................................................................................................................Finished!

In [7]:
mlp_forest.train(train_data, window_size)

training...........................................................................................................................................................................................................Finished!

In [20]:
offset = [None for i in range(window_size)]
mlp_pred = mlp_forest.predict(x) + offset
dt_pred = dt_forest.predict(x) + offset

In [9]:
def predict_unseen(forest, initial_window, n_predictions) -> list:
    complete_pred = initial_window
    window = initial_window
    for i in range(n_predictions):
        current_predictions = []
        for model in forest:
            pred = model.predict([window])
            current_predictions.append(pred[0])
        avg_pred = sum(current_predictions) / len(current_predictions)
        window = window[1:]
        window.append(avg_pred)
        complete_pred.append(avg_pred)
    return complete_pred


In [10]:
# SIMON
def predict_avg_of_combined(trees, slide):
    summed = []
    for i in range(len(trees)):
        summed.append(trees[i].predict(slide).item())
    return round(mean(summed), 2)

In [11]:
initial_window = np_2021[-10:].reshape(1, -1).tolist()
initial_window

[[123.64, 113.5, 99.96, 83.23, 114.01, 131.72, 201.82, 156.86, 152.42, 147.86]]

In [12]:
a = np_2022.tolist()
for i in range(365 - len(np_2022) - 1):
    b = predict_avg_of_combined(dt_forest.forest, initial_window)
    a.append(b)

In [17]:
#dt_pred_unseen = predict_unseen(dt_forest.forest, initial_window, 365 - len(np_2022))
#dt_pred_2022 = np_2022.tolist() + dt_pred_unseen

ValueError: Found array with dim 3. DecisionTreeRegressor expected <= 2.

In [14]:
# initial_window = np_2021[-10:].tolist()

In [19]:
 mlp_pred_unseen = predict_unseen(mlp_forest.ensemble, initial_window, 365 - len(np_2022))
mlp_pred_2022 = np_2022.tolist() + mlp_pred_unseen

ValueError: Found array with dim 3. MLPRegressor expected <= 2.

#### 5c)
*Use matplotlib to plot the predictions for the remainder of 2022 (5 pts)*

In [16]:
plt.plot(dt_pred_2022, mlp_pred_2022)

NameError: name 'dt_pred_2022' is not defined